#Outline of Project

##Dashboard as a Communication Tool

Here, we write some hypotheses and predictions based on work we've read in class about the efficacy of communication and rapid/accurate data transfer between agents to accomplish a task effexctively
 

##Dashboard as a Fire Modelling and Recommendation Tool

Here we outline the way our dashboard works to model a fire based on some data and inputs and the ways in which we can model extinguishing this fire and which team methods work better for this purpose

##Putting it all together

Combining these two components in the paper, we can essentialyl draw the conclusion that by improving communication and record-keeping between jumpers and dispatch, and by also being able to provide solid real-time recommendations, we can utilize multi-agent interactions coumpled with a powerful fire model to improve the effectiveness fo wildfire fighting operations by smokejumpers.

##Sequence of iterations to final product
Mine any necessary or valuable data
Inputs to dispatch:
   - Location of fire
   - Vegetation
       - Type
       - Flammability
       - Weighted avg of all types of fuel
       - Amount (volume) of fuel
   - Size/shape of fire
       - Mark more valuable/useful coordinates through which the fire passes
       - Main points of the fire
       - Where is the anchor point?
       - 2D/3D grid of where the fire is
   - Wind estimate
   - Conditions estimate (dry, wet, etc)
   - Rate of speed?
   - Sensitive burning areas
       - Cost estimates (if we're really fancy)

Find a model that we actually want to use so that we know which of these we need and **in what format** (ie how to we store shape and size of the fire?)

Outputs from dispatch:
   - Temporal estimate of what the fire will look like look like at given time intervals (30m/1hr/2hr)
   - Will the anchor move? Will it spread in a shape that requires more detailed information
   - Update the grid of the fire if it exists
   - Where to target the fire and how many people per target
   - Stupid extension stuff:
       - Calculate predicted cost? (if we're fancy)
       - Compute a danger evaluation (Scale 1-100 of urgency or something)
       - Calculate distance to base and approx flight time
       - Provide a resources recommendation (# of fighters, # of planes, amt of fuel, level of IC)
    
Create a class that collects inputs from dispatch

Create some function/class that can take 1-many variables and spt out some information about a fire

Dashboard that takes in user input and returns a fire/fire spread model

Graphical Dashboard

###Evaluation and means of testing

Given some model of the fire, most likely derived from the temperature paper, we want to make some prediction as to the best arrangement of smokejumpers to most efficiently handle the fire.

This can be done by making each fighter a hypothetical "cooling point" and modeling what happens to the fire, essentially determining how quickly it will be put out.

We can determine these cooling pts automatically by trying a bunch of combinations and finding the best one.

This doesn't take into account safety of firefighters, so we should implement a safety rating that is represented by a negative reward (anything to inform realistic landing areas).

On top of this, we can create 2-3 heuristic approaches to fire tackling (attack the anchor, attack the flanks, cluster together in one location, or spread out in and/or around the fire [email someone to find out possible approaches of attack])

Using these heuristic approaches, we can take a couple of verieties of fire (big, small, long, round, multi-point, etc [look into finding realistic approximations]), and perhaps even getting actual fires that were recorded in alaska and using their dimensions or any info we can get on them.

Then we can run the heuristic approaches on the different types of fires (based on our spread model) and evaluate how quickly the fire gets put out in each scenario and how far it spreads in the time it takes to put it out

###Timeline

####Coding component

[by Apr 23] Using this paper: http://arxiv.org/pdf/0709.0086.pdf, create a Python class that is able to take fire inputs (assuming we have them) and spit out some desired output. 

[by Apr 25] Using not real, toy model data, make sure this model works and simulate a cooling point to see if we can actually extinguish a fire

[by Apr 27] Create a presentation based on our approach, the findings above, our current prgress with the model, what we plan to do (port data, test cases, etc), and an outline of the paper

[by Apr 30] Should have all of our test cases done and even tested on the toy/fake data to see if we can gather valuable results from this

[by May 5] Port real data into this. Port in the vegetation and wind data and actually compute the A, B, C coeffs, and essentially have the entire coding portion completely done. Any visualization code and the actual GUI for dashboard (w/e i is) should be done by this point as well

[by May 7] Finish the final writeup and submit


####Paper component


## Go Time

In [1]:
import numpy as np

In [5]:
class FireModel:
    def __init__(self, fireLayerTemp, fuelSupplyMass, thermDiffus, tempRise, propCoef, scaledCoef, fuelDispearRate, ambTemp, windSpeed):
        self.fireLayerTemp = fireLayerTemp
        self.fuelSupplyMass = fuelSupplyMass
        self.thermDiffus = thermDiffus
        self.tempRise = tempRise
        self.propCoef = propCoef
        self.scaledCoef = scaledCoef
        self.fuelDispearRate = fuelDispearRate
        self.ambTemp = ambTemp
        self.windSpeed = windSpeed

In [8]:
def changeTemp():
    diffusion = np.gradient(self.thermDiffus * np.gradient(self.fireLayerTemp))
    heatAdvancedByWind = self.windSpeed * np.gradient(self.fireLayerTemp)
    rateFuelComsumedByBurning = self.fuelSupplyMass * np.exp(-self.propCoef/(self.fireLayerTemp - self.ambTemp))
    convectiveHeatLostAtmosphere = self.scaledCoef * (self.fireLayerTemp - self.ambTemp)
    return diffusion - heatAdvancedByWind + self.tempRise * (rateFuelComsumedByBurning - convectiveHeatLostAtmosphere)


In [9]:
def changeFuelSupply():
    expTemp = np.exp(-self.propCoef/(self.fireLayerTemp - self.ambTemp))
    return -1 * self.fuelDispearRate * self.fuelSupplyMass * expTemp

In [ ]:
# Constants
k = 0.21360 #thermDiffus
A = 187.93 #tempRise
B = 558.49 #propCoeff
C = .000048372 #scaledCoeff
Cs = .1625 #fuelDispearRate
Tc = 1200 # Maximum Stable Combustion Temperature
Ti = 670 # Unstable Equilibrium Point

# Data-dependent variables
T = 10. #fireLayerTemp
S = 0.3 #fuelSupplyMass
Ta = 189 #ambTemp
v = 17.83 #windSpeed

## Cellular Automata

http://en.wikipedia.org/wiki/Forest-fire_model

In [100]:
import random

In [205]:
import random

## class representation for each cell
class Cell:
    def __init__(self, x, y, time, veg_inten, wind_direc, wind_inten, fire_inten,firefighter, ff_info):
        self.x = x
        self.y = y
        self.time = time
        self.veg_inten = veg_inten # vegetation intensity (assuming the most intense, the more flammable; 0 means it can no longer catch on fire)
        self.wind_direc = wind_direc # wind direction
        self.wind_inten = wind_inten # wind intensity
        self.fire_inten = fire_inten # if there is a fire in the cell, how intensly is it burning
        self.firefighter = firefighter
        self.ff_info = ff_info ## but will be class firefighter
        
## class representation for each firefighter
class FireFighter:
    def __init__(self, cell, efficacy = 1):
        self.cell = cell # The starting cell/cell we're currently in
        self.path = [] # The path we've traverse so far
        self.efficacy = efficacy # Perhaps make this model vary on fighter skill level
        self.actList = range(8) # 0 = up left, ... 7 = left

    def bestAction(self):
        actList = [(-1,0),(0,-1),(0,1),(1,0)]
        act = random.choice(actList)
        self.path.append(act)
        return act
       
class AreaSimulation:
    def __init__(self, L):
        self.grid = {}
        self.hood = ((-1,-1), (-1,0), (-1,1),
                (0,-1),          (0, 1),
                (1,-1),  (1,0),  (1,1))
        
        self.L = L
        self.time = 0
        
    def fight_fire(self,x,y,ff_info):
        self.grid[(x,y)].firefighter = True
        self.grid[(x,y)].ff_info = ff_info
        return
        
    ## rewrite this
    def initialize(self):
        for x in range(self.L):
            for y in range(self.L):
                self.grid[(x,y)] = Cell(
                    x= x,
                    y= y,
                    time= 0,
                    veg_inten= random.random(),
                    wind_direc= "right", ### not using this for now
                    wind_inten= random.random(),
                    fire_inten= 0.00,
                    firefighter = False,
                    ff_info = None 
                )
        ### now I'm going to "start" a fire in the upper left corner
        self.grid[(1,2)].fire_inten = .55
        self.grid[(1,1)].fire_inten = .60
        self.grid[(2,1)].fire_inten = .31
        
        return self.grid
    
    def gnew(self):
        newgrid = {}
        new_ff_coord = []
        self.time += 1
        # iterate through all the cells
        for x in range(self.L):
            for y in range(self.L): 
#                 if self.grid[(x,y)].veg_inten == 0: # no fire is possible here
#                     newgrid[(x,y)] = self.grid[(x,y)] ## need to update time
#                     newgrid[(x,y)].time +=1
#                     newgrid[(x,y)].fire_inten = 0
                if self.grid[(x,y)].firefighter == True: ## there is a firefighter in that cell
                    print "fire fighter coord", x,y
                    newgrid[(x,y)] = Cell(
                        x= x,
                        y= y,
                        time = self.time,
                        veg_inten= self.grid[(x,y)].veg_inten, # vegetation preserved
                        wind_direc= self.grid[(x,y)].wind_direc,
                        wind_inten= self.grid[(x,y)].wind_inten,
                        fire_inten= 0.0, # fire put out
                        firefighter = False, ### since firefighter leaves the cell
                        ff_info = None 
                   )
                    # append that we have traversed 
                    self.grid[(x,y)].ff_info.path.append((x,y))
                    ## pick where the fire firefighter will be in the next step
#                     action = self.grid[(x,y)].ff_info.bestAction() ## somewhere here need to check if it's a legal action
                    
#                     # there should be a way to get this at the end
#                     newgrid[(x+action[0], y+action[1])].firefighter = True
#                     newgrid[(x+action[0], y+action[1])].ff_info = self.grid[(x,y)].ff_info
                    new_ff_coord.append((x,y))
                ## TO DO: indicate here which cell the firefigher will go to next
                elif self.grid[(x,y)].fire_inten > 0: # cell is burning
                    new_fire_inten = min(self.grid[(x,y)].fire_inten *self.grid[(x,y)].veg_inten * 3,1) ### take into account vegetation
                    newgrid[(x,y)] = Cell(
                        x= x,
                        y= y,
                        time = self.time if not self.grid[(x,y)].time else self.grid[(x,y)].time,
                        veg_inten= max(0,self.grid[(x,y)].veg_inten -.01), ## figure out a better way to decay
                        wind_direc= self.grid[(x,y)].wind_direc,
                        wind_inten= self.grid[(x,y)].wind_inten,
                        # some previous intensity and a combination of surrounding intensities 
                        fire_inten= round(new_fire_inten,2),
                        firefighter = False,
                        ff_info = None 
                   )
                elif self.grid[(x,y)].fire_inten == 0: # cell is not burning but can catch (assuming cells don't randomly catch fire)
                    ## need to take into account the conditions of surrounding cells
                    total_inten = 0
                    for dx,dy in self.hood:
                        if (x+dx,y+dy) in self.grid:
                            total_inten += self.grid[(x+dx,y+dy)].fire_inten # will somehow need to handle the surrounding fire  
                    new_fire_inten = total_inten/8 * self.grid[(x,y)].veg_inten # to do: take into account wind somehow
                    newgrid[(x,y)] = Cell(
                        x= x,
                        y= y,
                        time= self.grid[(x,y)].time,
                        veg_inten= self.grid[(x,y)].veg_inten, ## figure out a better way to decay
                        wind_direc= self.grid[(x,y)].wind_direc,
                        wind_inten= self.grid[(x,y)].wind_inten,
                        fire_inten= round(new_fire_inten,2),
                        firefighter = False,
                        ff_info = None 
                   )
        for x,y in new_ff_coord:
            action = self.grid[(x,y)].ff_info.bestAction() ## somewhere here need to check if it's a legal action
            newgrid[(x+action[0], y+action[1])].firefighter = True
            newgrid[(x+action[0], y+action[1])].ff_info = self.grid[(x,y)].ff_info
        self.grid = newgrid
        return newgrid

    ## printing functions
    def gprint(self):
        txt = '\n'.join(' * '.join(str(self.grid[(x,y)].fire_inten) for x in range(L))
                         for y in range(L))
        print(txt)

In [207]:
sim = AreaSimulation(15)
sim.initialize()
iters = 8
for i in range(iters):
    #sim.quickprint(grid)
    print "iteration",i
    if i == 3:
        # introduce some firefighters
        ff1 = FireFighter((3,2))
        sim.fight_fire(2,1,ff1)
    sim.gprint()
    sim.gnew()

iteration 0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.6 * 0.31 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.55 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
iteration 1
0.07 * 0.11 * 0.04 * 0.01 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.08 * 1.0 * 0.55 * 0.01 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.01 * 0.44 * 0.08 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.01 * 0.0 * 0.04 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0
0.0 * 0.0 * 0.0 * 0.0 * 0.0 * 0.0 